<a href="https://colab.research.google.com/github/MilizCom/Machine-Learning/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#import library yang diperlukan
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.metrics import accuracy_score

In [22]:
df = {
    'Umur': ['<=30', '<=30', '30...40', '>40', '>40', '<=30', '31...40', '<=30', '<=30', '>40', '<=30', '31...40', '31...40', '>40'],
    'Pendapatan': ['tinggi', 'tinggi', 'tinggi', 'sedang', 'rendah', 'rendah', 'rendah', 'sedang', 'rendah', 'sedang', 'sedang', 'sedang', 'tinggi', 'sedang'],
    'Mhs': ['bukan', 'bukan', 'bukan', 'bukan', 'ya', 'ya', 'ya', 'bukan', 'ya', 'ya', 'ya', 'bukan', 'ya', 'bukan'],
    'Rating Kredit': ['fair', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'fair', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'excellent'],
    'Beli Komputer': ['tdk', 'tdk', 'ya', 'ya', 'ya', 'tdk', 'ya', 'tdk', 'ya', 'ya', 'ya', 'ya', 'ya', 'tdk']
}
df = pd.DataFrame(df)
print(df)

       Umur Pendapatan    Mhs Rating Kredit Beli Komputer
0      <=30     tinggi  bukan          fair           tdk
1      <=30     tinggi  bukan     excellent           tdk
2   30...40     tinggi  bukan          fair            ya
3       >40     sedang  bukan          fair            ya
4       >40     rendah     ya          fair            ya
5      <=30     rendah     ya     excellent           tdk
6   31...40     rendah     ya          fair            ya
7      <=30     sedang  bukan          fair           tdk
8      <=30     rendah     ya          fair            ya
9       >40     sedang     ya          fair            ya
10     <=30     sedang     ya     excellent            ya
11  31...40     sedang  bukan     excellent            ya
12  31...40     tinggi     ya          fair            ya
13      >40     sedang  bukan     excellent           tdk


In [23]:
le = LabelEncoder()
encoded_df = df.copy()
for col in encoded_df.columns:
    encoded_df[col] = le.fit_transform(df[col])
print(encoded_df)

    Umur  Pendapatan  Mhs  Rating Kredit  Beli Komputer
0      2           2    0              1              0
1      2           2    0              0              0
2      0           2    0              1              1
3      3           1    0              1              1
4      3           0    1              1              1
5      2           0    1              0              0
6      1           0    1              1              1
7      2           1    0              1              0
8      2           0    1              1              1
9      3           1    1              1              1
10     2           1    1              0              1
11     1           1    0              0              1
12     1           2    1              1              1
13     3           1    0              0              0


In [24]:
label_counts = encoded_df['Beli Komputer'].value_counts()
total_samples = len(encoded_df)
prior_prob = label_counts / total_samples

In [27]:
feature_probs = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
features = ['Umur', 'Pendapatan', 'Mhs', 'Rating Kredit']

target = 'Beli Komputer'

for feature in features:
    for value in encoded_df[feature].unique():
        for class_value in encoded_df[target].unique():
            num = len(encoded_df[(encoded_df[feature] == value) & (encoded_df[target] == class_value)])
            den = label_counts[class_value]
            feature_probs[feature][value][class_value] = num / den if den != 0 else 0

print("Probabilitas Prior:")
print(prior_prob)
print("\nProbabilitas Likelihood:")
for feature in features:
  print(f"\n{feature}:")
  for value in feature_probs[feature]:
    for class_value in feature_probs[feature][value]:
      print(f"({feature}={value}| Kelas={class_value})={feature_probs[feature][value][class_value]:.4f}")

Probabilitas Prior:
Beli Komputer
1    0.642857
0    0.357143
Name: count, dtype: float64

Probabilitas Likelihood:

Umur:
(Umur=2| Kelas=0)=0.8000
(Umur=2| Kelas=1)=0.2222
(Umur=0| Kelas=0)=0.0000
(Umur=0| Kelas=1)=0.1111
(Umur=3| Kelas=0)=0.2000
(Umur=3| Kelas=1)=0.3333
(Umur=1| Kelas=0)=0.0000
(Umur=1| Kelas=1)=0.3333

Pendapatan:
(Pendapatan=2| Kelas=0)=0.4000
(Pendapatan=2| Kelas=1)=0.2222
(Pendapatan=1| Kelas=0)=0.4000
(Pendapatan=1| Kelas=1)=0.4444
(Pendapatan=0| Kelas=0)=0.2000
(Pendapatan=0| Kelas=1)=0.3333

Mhs:
(Mhs=0| Kelas=0)=0.8000
(Mhs=0| Kelas=1)=0.3333
(Mhs=1| Kelas=0)=0.2000
(Mhs=1| Kelas=1)=0.6667

Rating Kredit:
(Rating Kredit=1| Kelas=0)=0.4000
(Rating Kredit=1| Kelas=1)=0.7778
(Rating Kredit=0| Kelas=0)=0.6000
(Rating Kredit=0| Kelas=1)=0.2222


In [26]:
def predict(row):
  posterior_probs = {}
  for class_value in encoded_df[target].unique():
    posterior = prior_prob[class_value]
    for feature in features:
      feature_value = row[feature]
      posterior *= feature_probs[feature][feature_value][class_value]
    posterior_probs[class_value] = posterior
  return max(posterior_probs, key=posterior_probs.get)

In [19]:
  encoded_df['Prediksi'] = encoded_df.apply(predict, axis=1)
  correct_predictions = (encoded_df['Prediksi'] == encoded_df[target]).sum()
  accuracy = correct_predictions / total_samples
  print(f"\nAkurasi: {accuracy:.2%}")

  print(f"\nPrediksi vc Aktual:")
  print(encoded_df[['Beli Komputer', 'Prediksi']])


Akurasi: 85.71%

Prediksi vc Aktual:
    Beli Komputer  Prediksi
0               0         0
1               0         0
2               1         1
3               1         1
4               1         1
5               0         1
6               1         1
7               0         0
8               1         1
9               1         1
10              1         0
11              1         1
12              1         1
13              0         0
